# Caterpillar Tube Pricing
## Environment : Python 3
## Author : Arion

# import packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import time

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 1000)

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline 

# import data

In [2]:
train = pd.read_csv("../output/combination.csv")

# data preparation 

In [3]:
#There is no common tube_assembly_id between train and test data. So we drop this variable.
train.drop("tube_assembly_id", axis=1, inplace=True)

In [4]:
train.head().transpose()

,0,1,2,3,4
supplier,S-0066,S-0066,S-0066,S-0066,S-0066
quote_date,2013-07-07,2013-07-07,2013-07-07,2013-07-07,2013-07-07
annual_usage,0,0,0,0,0
min_order_quantity,0,0,0,0,0
bracket_pricing,Yes,Yes,Yes,Yes,Yes
quantity,1,2,5,10,25
cost,21.9059,12.3412,6.60183,4.68777,3.54156
material_id,SP-0019,SP-0019,SP-0019,SP-0019,SP-0019
diameter,6.35,6.35,6.35,6.35,6.35
wall,0.71,0.71,0.71,0.71,0.71


In [5]:
train.quote_date = pd.to_datetime(train.quote_date)

In [6]:
#add new numeric time features

train["year"] = train.quote_date.dt.year
train["month"] = train.quote_date.dt.month
train["day"] = train.quote_date.dt.day
train["day_of_week"] = train.quote_date.dt.dayofweek

In [7]:
#only use numeric data
data = train.select_dtypes(include=['int', 'float'])

In [8]:
#fill null by 0
data.replace(np.nan, 0, inplace=True)

# Utility Functions

In [9]:
#define a evaluation function

def rmsle_score(preds, true):
    rmsle_score = (np.sum((np.log1p(preds)-np.log1p(true))**2)/len(true))**0.5
    return rmsle_score

In [10]:
#Define a evaluation matrix 
from sklearn.metrics.scorer import make_scorer

RMSLE = make_scorer(rmsle_score)

In [11]:
# define a function for comparing predictions and true data.

def compare_result(preds, true):
    compare = pd.DataFrame({"test_id": true.index,
                           "real_cost": true,
                           "pred_cost": preds})
    compare = compare[["test_id", "real_cost", "pred_cost"]].reset_index(drop=True)
    
    compare["error_percent_(%)"] = np.abs(compare.real_cost - compare.pred_cost) / compare.real_cost * 100
    
    return compare

# application of machine learning model

### Simple Linear Regression

In [12]:
# split for machine learning model

train_data, valid_data = train_test_split(data, test_size = 0.3)

label = "cost"

data_labels = train_data.columns.tolist()
data_labels.remove(label)

train_df = train_data[data_labels]
valid_df = valid_data[data_labels]
train_label = train_data[label]
valid_label = valid_data[label]

In [13]:
train_df.head()

,annual_usage,min_order_quantity,quantity,diameter,wall,length,num_bends,bend_radius,num_boss,num_bracket,other,type_totals,component_totals,spec_totals,year,month,day,day_of_week
14436,25,1,1,101.60,1.65,94.0,2,101.60,0,0,0,0,0.0,2,2011,8,1,0
24877,1,0,50,9.52,0.89,89.0,5,19.05,0,0,0,2,4.0,0,2014,2,1,5
26478,0,0,120,7.94,0.71,100.0,5,25.40,0,0,0,1,2.0,2,2007,2,20,1
22956,0,0,2,9.52,0.89,43.0,3,25.40,0,0,0,2,4.0,0,2013,8,1,3
10428,0,0,250,19.05,1.65,48.0,3,50.80,0,0,0,3,3.0,0,2013,10,1,1


In [14]:
valid_df.head()

,annual_usage,min_order_quantity,quantity,diameter,wall,length,num_bends,bend_radius,num_boss,num_bracket,other,type_totals,component_totals,spec_totals,year,month,day,day_of_week
23264,26,0,3,9.52,0.89,175.0,8,19.05,0,0,0,2,4.0,0,2013,5,3,4
19412,201,1,1,50.80,1.65,43.0,2,50.80,0,0,2,1,2.0,0,2012,8,1,2
18622,1,0,1,9.52,1.24,101.0,5,19.05,0,0,0,2,4.0,0,2012,8,13,0
23242,10,1,1,63.50,1.65,78.0,2,152.40,0,0,0,0,0.0,0,2012,8,15,2
10421,0,0,1,19.05,1.65,48.0,3,50.80,0,0,0,3,3.0,0,2013,10,1,1


In [15]:
#Linear regression

from sklearn.linear_model import LinearRegression

start = time.time()
linear=LinearRegression()


label_log=np.log1p(train_label)

model=linear.fit(train_df, label_log)
linear_preds1=model.predict(valid_df)

linear_preds=np.expm1(linear_preds1)
        
rmsle_linear = rmsle_score(linear_preds, valid_label)
print ("Linear RMSLE is : {}".format(rmsle_linear))

compare_linear_log = compare_result(preds=linear_preds, true=valid_label)

end = time.time()
duration = end - start
print ("It takes {} seconds".format(duration))

Linear RMSLE is : 0.6509275370295581
It takes 0.1875917911529541 seconds


### Linear Regression with cross_val_score

In [16]:
# split for cross_val_score machine learning model

label = "cost"

data_labels = data.columns.tolist()
data_labels.remove(label)

X = data[data_labels]
y = data[label]

In [17]:
#Linear regression and cross_val_score
from sklearn.cross_validation import cross_val_score

from sklearn.linear_model import LinearRegression

start = time.time()
linear=LinearRegression()

rmsle_scores = cross_val_score(linear, X, y, scoring=RMSLE, cv=5)
print("RMSLE are:{}".format(rmsle_scores))
print("Mean RMSLE score:{}".format(np.mean(rmsle_scores)))

end = time.time()
duration = end - start
print ("It takes {} seconds".format(duration))

RMSLE are:[ 0.82270528  0.82014801  0.87274333  0.85288201  0.86124792]
Mean RMSLE score:0.8459453097834008
It takes 0.06710386276245117 seconds


### Simple RandomForest Regression

In [18]:
# RandomForest Regression 
from sklearn.ensemble import RandomForestRegressor

start = time.time()
rf=RandomForestRegressor(random_state=0)

label_log=np.log1p(train_label)

model=rf.fit(train_df, label_log)
rf_preds1=model.predict(valid_df)

rf_preds=np.expm1(rf_preds1)
        
rmsle_rf = rmsle_score(rf_preds, valid_label)
print ("Random Forest RMSLE is : {}".format(rmsle_rf))

compare_rf_log = compare_result(preds=rf_preds, true=valid_label)

end = time.time()
duration = end - start
print ("It takes {} seconds".format(duration))

Random Forest RMSLE is : 0.2786956433088274
It takes 0.856114387512207 seconds


### RandomForest and cross_val_score

In [19]:
#RandomForest Regression and cross_val_score
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestRegressor

start = time.time()
rf=RandomForestRegressor(random_state=0)

rmsle_scores = cross_val_score(rf, X, y, scoring=RMSLE, cv=5)
print("RMSLE are:{}".format(rmsle_scores))
print("Mean RMSLE score:{}".format(np.mean(rmsle_scores)))

end = time.time()
duration = end - start
print ("It takes {} seconds".format(duration))

RMSLE are:[ 0.39838117  0.36137671  0.42144332  0.51476111  0.39939352]
Mean RMSLE score:0.4190711670228226
It takes 5.045459985733032 seconds
